In [ ]:
%store -r
%store

# Desplegando Campañas y Filtros

In [ ]:
import time
from time import sleep
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
pd.set_option('display.max_colwidth', 200)  


from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Cree campañas <a class="anchor" id="create"> </a>

Una campaña es una versión de solución hosteada; un endpoint que puede consultar para obtener recomendaciones. 

Comencemos a implementar las campañas. 

### User Personalization

In [ ]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-anime-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print (userpersonalization_campaign_arn)

In [ ]:
userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print (userpersonalization_campaign_arn)

Si quieres agregar Items nuevos o que la campaña tome antiguos, debes agregar itemExplorationConfig. 

Mas información: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_campaign



### SIMS


In [ ]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-anime-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
sims_campaign_arn

### Personalized Ranking

In [ ]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-anime-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
rerank_campaign_arn

### View campaign creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Campaigns`.
* You will now see a list of all of the campaigns you created above, including a column with the status of the campaign. Once it is `Active`, your campaign is ready to be queried.

Or simply run the cell below to keep track of the campaign creation status.

In [ ]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

## Crear filtros <a class="anchor" id="interact"> </a>

Ahora que todas las campañas están implementadas y activas, podemos crear filtros. Se pueden crear filtros tanto para elementos como para eventos. Algunos casos de uso comunes de filtros en Video On Demand son:

- Filtros categóricos basados en los metadatos del artículo: a menudo, los metadatos del artículo tendrán información sobre el título, como género, palabra clave, año, década, etc. El filtrado de estos puede proporcionar recomendaciones dentro de esos datos, como películas de acción.

- Eventos: es posible que desee filtrar ciertos eventos y proporcionar resultados basados en esos eventos, como mover un título de una recomendación de "sugerencias para mirar" a una recomendación de "mirar de nuevo".

Veamos los metadatos del elemento y las interacciones del usuario, para que podamos tener una idea de qué tipo de filtros podemos crear.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(personalize_data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
items_df.head(10)
#interactions_df.head(10)

In [ ]:
full_genre_list = []
for g in items_df['GENRE']:
    full_genre_list += g.split('|')
genres, count  =  np.unique(full_genre_list, return_counts= True)

In [ ]:
conteo = pd.DataFrame({'genre':genres, 'count': count}).sort_values(by='count',ascending=False).set_index('genre')

In [ ]:
conteo.plot(kind='bar', figsize=(15,5));

In [ ]:
genres_to_filter = list(conteo.reset_index()[:9]['genre'])

In [ ]:
genres_to_filter

In [ ]:
meta_filter_arns = []

In [ ]:
filters = personalize.list_filters()

In [ ]:
for ff in filters['Filters']:
    meta_filter_arns.append(ff['filterArn'])

In [ ]:
# Iterate through Genres
for genre in genres_to_filter:
    # Start by creating a filter
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # Add the ARN to the list
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # Add the ARN to the list
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

In [ ]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns

In [ ]:
meta_filter_arns